In [24]:
import os
import re
import pandas as pd
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/steam-recommendation-nlp-dataset/train.csv
/kaggle/input/steam-recommendation-nlp-dataset/test.csv
/kaggle/input/steam-recommendation-nlp-dataset/game_overview.csv
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/imdb-review-dataset/imdb_master.csv


In [23]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [25]:
training_df = pd.read_csv('/kaggle/input/steam-recommendation-nlp-dataset/train.csv')
testing_df=  pd.read_csv('/kaggle/input/steam-recommendation-nlp-dataset/test.csv')

In [26]:


# df1 = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', delimiter="\t")
# df1 = df1.drop(['id'], axis=1)
# df1.head()

In [27]:
# df2 = pd.read_csv('/kaggle/input/imdb-review-dataset/imdb_master.csv',encoding="latin-1")
# df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
# df2.columns = ["review","sentiment"]
# df2.head()

In [28]:
# df2 = df2[df2.sentiment != 'unsup']
# df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})
# df2.head()

In [29]:
# df = pd.concat([df1, df2]).reset_index(drop=True)
# df.head()

In [31]:
training_df.drop(["review_id","title","year"],axis=1,inplace=True)

In [32]:
training_df.head()

,user_review,user_suggestion
0,I'm scared and hearing creepy voices. So I'll...,1
1,"Best game, more better than Sam Pepper's YouTu...",1
2,"A littly iffy on the controls, but once you kn...",1
3,"Great game, fun and colorful and all that.A si...",1
4,Not many games have the cute tag right next to...,1


In [35]:
def rep(text):
    text = re.sub('♥♥♥♥',"worst bad horrible game",text)
    return text

training_df['user_review']=training_df.user_review.apply(rep)
testing_df['user_review']=testing_df.user_review.apply(rep)

def low(text):
    return text.lower()

training_df['user_review']=training_df.user_review.apply(low)
testing_df['user_review']=testing_df.user_review.apply(low)

def asc(text):
    text = re.sub('[^a-zA-Z]'," ",text)
    return text

training_df['user_review']=training_df.user_review.apply(asc)
testing_df['user_review']=testing_df.user_review.apply(asc)

In [37]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

training_df['Processed_Reviews'] = training_df.user_review.apply(lambda x: clean_text(x))
testing_df['Processed_Reviews'] = testing_df.user_review.apply(lambda x: clean_text(x))


In [45]:
testing_df['Processed_Reviews'] = testing_df.user_review.apply(lambda x: clean_text(x))

In [38]:
training_df.head()

,user_review,user_suggestion,Processed_Reviews
0,i m scared and hearing creepy voices so i ll...,1,scar hear creepy voice pause moment write re...
1,best game more better than sam pepper s youtu...,1,best game better sam pepper youtube account ...
2,a littly iffy on the controls but once you kn...,1,littly iffy control know play easy master m...
3,great game fun and colorful and all that a si...,1,great game fun colorful side note though ge...
4,not many games have the cute tag right next to...,1,many game cute tag right next horror tag steam...


In [39]:
training_df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

103.75471590259518

In [61]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 2000 #6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(training_df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(training_df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = training_df['user_suggestion']

embed_size = 103 ###############################
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 13995 samples, validate on 3499 samples
Epoch 1/3
13995/13995 [==============================] - 50s 4ms/step - loss: 0.5602 - acc: 0.7131 - val_loss: 0.4910 - val_acc: 0.7777
Epoch 2/3
13995/13995 [==============================] - 47s 3ms/step - loss: 0.3690 - acc: 0.8529 - val_loss: 0.5180 - val_acc: 0.7679
Epoch 3/3
13995/13995 [==============================] - 47s 3ms/step - loss: 0.3141 - acc: 0.8798 - val_loss: 0.5049 - val_acc: 0.7802


In [62]:
# df_test=pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip",header=0, delimiter="\t", quoting=3)
# df_test.head()
testing_df.head()

,review_id,title,year,user_review,Processed_Reviews
0,1603,Counter-Strike: Global Offensive,2015.0,naturally film main theme mortality nostalgia ...,naturally film main theme mortality nostalgia ...
1,1604,Counter-Strike: Global Offensive,2018.0,movie disaster within disaster film full great...,movie disaster within disaster film full great...
2,1605,Counter-Strike: Global Offensive,2018.0,movie kid saw tonight child love one point kid...,movie kid saw tonight child love one point kid...
3,1606,Counter-Strike: Global Offensive,2015.0,afraid dark leave impression several different...,afraid dark leave impression several different...
4,1607,Counter-Strike: Global Offensive,2015.0,accurate depiction small time mob life film ne...,accurate depiction small time mob life film ne...


In [63]:
# testing_df["user_review"]=df_test.review.apply(lambda x: clean_text(x))
# df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
# y_test = df_test["sentiment"]#######################################################################################
list_sentences_test = testing_df["Processed_Reviews"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
# from sklearn.metrics import f1_score, confusion_matrix
# print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
# print('Confusion matrix:')
# confusion_matrix(y_pred, y_test)

In [64]:
y_pred = y_pred.astype(int)

In [65]:
submission=pd.DataFrame(y_pred)
sub = pd.read_csv('/kaggle/input/steam-recommendation-nlp-dataset/test.csv')
submission['review_id']=sub['review_id']
submission= submission[['review_id',0]]
submission.rename(columns = {0:'user_suggestion'}, inplace = True)
submission.to_csv('lstm.csv', index=False)

In [66]:
submission.head(4)

,review_id,user_suggestion
0,1603,1
1,1604,0
2,1605,0
3,1606,1
